In [ ]:
/* Comment this out to disable prints and save space */
#define BLYNK_PRINT Serial

#define BLYNK_TEMPLATE_ID "TMPL6LNxDdJLz"
#define BLYNK_TEMPLATE_NAME "IoT Energy Meter"
#define BLYNK_AUTH_TOKEN "0z3W5PzrilHbjuFyuttlbWdxw7oPiAoW"

#include <Wire.h>
#include <LiquidCrystal_I2C.h>

// Set the LCD dimensions (16x2)
LiquidCrystal_I2C lcd(0x27, 16, 2);


#include <WiFi.h>
#include "time.h"
#include <WiFiClient.h>
#include <BlynkSimpleEsp32.h>

#include <HTTPClient.h>

// Your WiFi credentials.
// Set password to "" for open networks.
char ssid[] = "meter";
char pass[] = "12121212";


unsigned long previousMillis = 0;  // Variable to store the previous time
const long interval = 20000;       // Interval in milliseconds (1 second)

#include "ACS712.h"
#include <ZMPT101B.h>
#include <Wire.h>
#include <Adafruit_SSD1306.h>

ACS712 ACS(34, 3.7, 4095, 100);  // 125
ZMPT101B voltageSensor(35, 50.0);

#define SCREEN_WIDTH 128
#define SCREEN_HEIGHT 64
#define OLED_RESET -1

Adafruit_SSD1306 display(SCREEN_WIDTH, SCREEN_HEIGHT, &Wire, OLED_RESET);

float kwh;

int v, c, w;


#define switchOne 5
#define switchTwo 18
#define switchThree 19

#define switchOneIn 13
#define switchTwoIn 12
#define switchThreeIn 14



#define buzzer 4

BLYNK_WRITE(V0) {
  Serial.println(param.asInt());
  if (param.asInt() == 0) {
    digitalWrite(switchOne, HIGH);
  } else {
    digitalWrite(switchOne, LOW);
  }
}

BLYNK_WRITE(V1) {
  Serial.println(param.asInt());
  if (param.asInt() == 0) {
    digitalWrite(switchTwo, HIGH);
  } else {
    digitalWrite(switchTwo, LOW);
  }
}

BLYNK_WRITE(V2) {
  Serial.println(param.asInt());
  if (param.asInt() == 0) {
    digitalWrite(switchThree, HIGH);
  } else {
    digitalWrite(switchThree, LOW);
  }
}


BLYNK_CONNECTED() {
  Blynk.syncVirtual(V2);
  Blynk.syncVirtual(V3);
  Blynk.syncVirtual(V4);
}


int lastButtonStateOne = 1;    // previous state of the button
int lastButtonStateTwo = 1;    // previous state of the button
int lastButtonStateThree = 1;  // previous state of the button


bool one = true;
bool two = true;
bool three = true;

#define Gas 33

BlynkTimer timer;

void setup() {
  Serial.begin(115200);

  // Configure pin 33 as an input for the gas sensor
  pinMode(Gas, INPUT);

  pinMode(switchOne, OUTPUT);
  pinMode(switchTwo, OUTPUT);
  pinMode(switchThree, OUTPUT);

  pinMode(buzzer, OUTPUT);

  digitalWrite(switchOne, HIGH);
  digitalWrite(switchTwo, HIGH);
  digitalWrite(switchThree, HIGH);


  pinMode(switchOneIn, INPUT);
  pinMode(switchTwoIn, INPUT);
  pinMode(switchThreeIn, INPUT);

  Serial.println(__FILE__);
  Serial.print("ACS712_LIB_VERSION: ");
  Serial.println(ACS712_LIB_VERSION);

  ACS.autoMidPoint();
  Serial.print("MidPoint: ");
  Serial.println(ACS.getMidPoint());
  Serial.print("Noise mV: ");
  Serial.println(ACS.getNoisemV());

  voltageSensor.setSensitivity(500.0f);


  lcd.begin();
  lcd.backlight();  // Turn on the backlight (if available on your display)


  // Print a message on the LCD
  lcd.setCursor(0, 0);  // Set the cursor to the first column and first row
  lcd.print("IoT Energy Meter");


  Blynk.begin(BLYNK_AUTH_TOKEN, ssid, pass);


  timer.setInterval(30000L, sendSensorData);  // Call every 30 seconds
}

void loop() {

  Blynk.run();

  timer.run();


  // Read the analog value from pin 33
  int sensorValue = analogRead(33);

  // Print the analog value to the Serial Monitor
  Serial.print("Gas Sensor Value: ");
  Serial.println(sensorValue);

  if (sensorValue > 500) {
    Blynk.logEvent("alert", "Gas Detect");

    lcd.clear();

    // Print a message on the LCD
    lcd.setCursor(2, 0);  // Set the cursor to the first column and first row
    lcd.print("Gas Detected");


    // Make the buzzer beep 5 times
    for (int i = 0; i < 5; i++) {
      digitalWrite(buzzer, HIGH);  // Turn the buzzer on
      delay(200);                  // Wait for 500 milliseconds
      digitalWrite(buzzer, LOW);   // Turn the buzzer off
      delay(200);                  // Wait for 500 milliseconds
    }

    // Add a delay before repeating (if needed)
    delay(2000);  // Wait for 2 seconds before looping again (optional)
  }



  int buttonStateOne = digitalRead(switchOneIn);
  if (buttonStateOne == HIGH) {
    digitalWrite(buzzer, HIGH);
    delay(200);
    digitalWrite(buzzer, LOW);
  }
  // Check for a button press
  if (buttonStateOne != lastButtonStateOne) {
    if (buttonStateOne == LOW) {
      //  delay(10);
      Serial.println("Button Click");

      if (one == true) {
        Serial.println("LOW");
        one = false;
        digitalWrite(switchOne, HIGH);
        Blynk.virtualWrite(V0, LOW);
      } else {
        Serial.println("HIGH");
        one = true;
        digitalWrite(switchOne, LOW);
        Blynk.virtualWrite(V0, HIGH);
      }
    }
    delay(10);  // Debounce delay
  }
  lastButtonStateOne = buttonStateOne;



  int buttonStateTwo = digitalRead(switchTwoIn);
  if (buttonStateTwo == HIGH) {
    digitalWrite(buzzer, HIGH);
    delay(200);
    digitalWrite(buzzer, LOW);
  }
  // Check for a button press
  if (buttonStateTwo != lastButtonStateTwo) {
    if (buttonStateTwo == LOW) {
      // delay(10);
      Serial.println("Button Click");
      if (two == true) {
        Serial.println("LOW");
        two = false;
        digitalWrite(switchTwo, HIGH);
        Blynk.virtualWrite(V1, LOW);
      } else {
        Serial.println("HIGH");
        two = true;
        digitalWrite(switchTwo, LOW);
        Blynk.virtualWrite(V1, HIGH);
      }
    }
    delay(10);  // Debounce delay
  }
  lastButtonStateTwo = buttonStateTwo;






  int buttonStateThree = digitalRead(switchThreeIn);
  if (buttonStateThree == HIGH) {
    digitalWrite(buzzer, HIGH);
    delay(200);
    digitalWrite(buzzer, LOW);
  }
  // Check for a button press
  if (buttonStateThree != lastButtonStateThree) {
    if (buttonStateThree == LOW) {
      //delay(10);
      Serial.println("Button Click");
      if (three == true) {
        Serial.println("LOW");
        three = false;
        digitalWrite(switchThree, HIGH);
        Blynk.virtualWrite(V2, LOW);
      } else {
        Serial.println("HIGH");
        three = true;
        digitalWrite(switchThree, LOW);
        Blynk.virtualWrite(V2, HIGH);
      }
    }
    delay(10);  // Debounce delay
  }
  lastButtonStateThree = buttonStateThree;






  float average = 0;
  for (int i = 0; i < 100; i++) {
    average += ACS.mA_AC();
  }
  float mA = average / 100.0;

  if (mA > 200) {

    c = mA;

    Serial.print("Current: ");
    Serial.println(c);
  } else {
    Serial.print("Current: ");
    Serial.println(mA);
    c = 0;
  }

  current = ACS.mA_AC();


  float voltage = voltageSensor.getRmsVoltage();
  if (voltage > 20) {

    v = voltage;

    Serial.print("voltage: ");
    Serial.println(v);
  } else {
    v = 0;
  }


  currentInAmps = current / 1000.0;

P = voltage * currentInAmps * 0.9; //0.9 pf value

float  E = 0;

	for(int i=0;i<50;i++)
	{
		E += P * (20/1000);
	}


double kWh = E / (3600* 1000); // kwh calculating

  delay(1000);



  unsigned long currentMillis = millis();  // Get the current time

  // Check if the time interval has passed
  if (currentMillis - previousMillis >= interval) {
    // Update the previous time for the next interval
    previousMillis = currentMillis;

    if (kwh >= 0.05) {
      Blynk.logEvent("alert", "Reach energy Limits");
      for (int i = 0; i < 5; i++) {
        digitalWrite(buzzer, HIGH);  // Play tone at 1000 Hz
        delay(200);                  // Duration of the tone in milliseconds
        digitalWrite(buzzer, LOW);   // Stop the tone
        delay(200);                  // Delay between tones
      }
    }

    String tk;
    String kwhS;

    // Your code to execute every interval goes here
    //Blynk.virtualWrite(V0, v);
    // Blynk.virtualWrite(V1, c);
    //Blynk.virtualWrite(V2, w);
    if (kwh > 0) {
      Serial.print("kWh: ");
      Serial.println(kwh);
      kwhS = String(kwh,4) + "kWh"; //4 decimal point
      Blynk.virtualWrite(V3, kwhS);
      tk = String(kwh * 7) + "Tk";
      Blynk.virtualWrite(V4, tk);
    } else {
      tk = "0.00Tk";
      kwhS = "0.00kWh";
    }

    lcd.clear();


    int quotient = c / 1000;   // Get the integer part of the quotient
    int remainder = c % 1000;  // Get the remainder

    // Extract the first two digits of the remainder
    int firstDecimal = remainder / 100;         // Get the first digit after the decimal
    int secondDecimal = (remainder / 10) % 10;  // Get the second digit after the decimal

    // Format the result to always show two decimal places
    String Am = String(quotient) + "." + String(firstDecimal) + String(secondDecimal);

    // Output the result
    Serial.println(Am);  // It will print "12.34"



    // Print a message on the LCD
    lcd.setCursor(0, 0);  // Set the cursor to the first column and first row
    lcd.print("U:");
    lcd.print(kwhS);
    lcd.print(" ");
    lcd.setCursor(11, 0);
    lcd.print(Am);
    lcd.print("A");


    lcd.setCursor(0, 1);  // Set the cursor to the first column and first row
    lcd.print("B:");
    lcd.print(tk);
    lcd.print(" ");
    lcd.setCursor(12, 1);
    lcd.print(v);
    lcd.print("V");
  }
}

void sendSensorData() {
  String url = "https://script.google.com/macros/s/AKfycby2c4cCMx00f20gbBcztFzO34XB5iyuzJa1qELhkzNYf-U29R-t4UaufoWTxaCdydPS/exec?";
  url += "voltage=" + String(v);
  url += "&current=" + String(c / 1000.0, 2);
  //url += "&units=" + String(w);
  url += "&kwh=" + String(kwh,4);
  url += "&bill=" + String(kwh * 7);
  url += "&gas_value=" + String(analogRead(Gas));

  HTTPClient http;
  http.begin(url);
  int httpResponseCode = http.GET();
  if (httpResponseCode > 0) {
    Serial.println("HTTP Response: " + http.getString());
  } else {
    Serial.println("Error on HTTP request");
  }
  http.end();
}
